In [1]:
!pip install numpy==1.26.4
!pip install sentence-transformers==2.2.2
!pip install transformers==4.40.1 huggingface_hub==0.22.2
!pip install TTS==0.22.0
!pip install -r https://raw.githubusercontent.com/coqui-ai/TTS/dev/requirements.txt
!pip install faiss-cpu==1.7.4
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q fastapi==0.100.0 uvicorn==0.22.0 python-multipart
!pip install telethon
!pip install gTTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 842.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-

In [2]:


# إزالة النسخة القديمة إن وُجدت
!rm -f ngrok ngrok.zip ngrok-v3.tgz

# تحميل أحدث نسخة من ngrok v3
!wget -q -O ngrok-v3.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

# فك الضغط
!tar -xvzf ngrok-v3.tgz


ngrok


In [3]:
!./ngrok version


ngrok version 3.23.1


In [4]:
# استبدل النص التالي بالتوكن الخاص بك من لوحة ngrok
!./ngrok config add-authtoken 2jWVfjzQtDSJm3YlDOQUoDV6Lz3_53ejySSv1WMG1jVWm14vz


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# **متنسااااااش تعمل Restart Session قبل ما تشغل الـ Cell الجاية**
# **آخر آخر حاجة**
# **Last Last Final**

In [ ]:
GROQ_API_KEY = "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"

In [ ]:
# اخر حاجة
from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from fastapi.responses import FileResponse
from pydantic import BaseModel, Field
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import uvicorn
import os
import whisper
import tempfile
from TTS.api import TTS as CoquiTTS
import nest_asyncio
import asyncio
import subprocess
import time
import json
from pathlib import Path
import httpx # استيراد المكتبة الجديدة

import base64
import io

from telethon.errors import SessionPasswordNeededError
from telethon.sessions import StringSession
from telethon.sync import TelegramClient

nest_asyncio.apply()

app = FastAPI()


# Ngrok setup (optional, comment out if not needed or run ngrok separately)
# Ensure ngrok executable is in the same directory or in PATH
try:
    ngrok_process = subprocess.Popen(["./ngrok", "http", "8000"])
    time.sleep(3) # Give ngrok some time to start
    # Try fetching the public URL from ngrok's API
    try:
        r = requests.get("http://localhost:4040/api/tunnels")
        r.raise_for_status() # Raise an exception for HTTP errors
        public_url = r.json()["tunnels"][0]["public_url"]
        print(f"ngrok public URL: {public_url}")
    except requests.exceptions.RequestException as e:
        print(f"Could not connect to ngrok API: {e}. Ngrok might still be running.")
        print("Check ngrok status manually if needed.")
    except (KeyError, IndexError) as e:
        print(f"Could not parse ngrok API response: {e}")
        print("ngrok JSON response:", r.text)
except FileNotFoundError:
    print("ngrok executable not found. Skipping ngrok startup.")
except Exception as e:
    print(f"An error occurred during ngrok setup: {e}")


# Global models (loaded once)
try:
    whisper_model = whisper.load_model("large-v3")
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2') # Renamed for clarity
    # Initialize one TTS model. The second one in original code would overwrite the first.
    # Using fast_pitch as it was the last one assigned.
    # tts_model = CoquiTTS(model_name="tts_models/en/ljspeech/fast_pitch")
    # tts_model = CoquiTTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=True)
    # tts_model = CoquiTTS(model_name="tts_models/en/ljspeech/tacotron2-DDC")
    # tts_model = CoquiTTS(model_name="tts_models/en/vctk/vits", progress_bar=True)
except Exception as e:
    print(f"Error loading one or more models: {e}")
    print("Please ensure model files are correctly downloaded and accessible.")
    # Potentially exit or disable features if models can't load
    # For now, we'll let it continue and it will fail later if these models are used.




GROQ_API_KEY = "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"

BASE_CHARACTER_DATA_PATH = Path("data_storage/characters")
BASE_CHARACTER_DATA_PATH.mkdir(parents=True, exist_ok=True) # Ensure base directory exists

# --- إعدادات تيليجرام (الخاصة بك كمطور) ---
API_ID = 24838737  # استبدل بـ api_id الخاص بك
API_HASH = 'dd8fe04fbfb8db4e7f0b8e379adc4971'  # استبدل بـ api_hash الخاص بك
pending_clients = {} # لحفظ جلسات تسجيل الدخول المؤقتة






# --- Pydantic Models ---
class CharacterProfileData(BaseModel):
    characterId: str
    characterName: str
    bio: str
    answers: dict

class WhatsAppMessage(BaseModel):
    id: str
    from_user: str = Field(..., alias='from')
    to: str
    timestamp: int
    body: str

class WhatsAppPayload(BaseModel):
    userId: str
    characterName: str
    characterId: str
    messages: list[WhatsAppMessage]

# --- Pydantic Models ---
class PhoneRequest(BaseModel):
    phone_number: str

class LoginRequest(BaseModel):
    phone_number: str
    code: str
    password: str | None = None

class ProcessChatRequest(BaseModel):
    chatId: int

# --- AI Helper Functions ---
async def call_groq_api(system_prompt: str, user_prompt: str) -> str:
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "meta-llama/llama-4-scout-17b-16e-instruct",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.3,
    }
    try:
        async with httpx.AsyncClient() as client:
            response = await client.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload, timeout=45.0)
            response.raise_for_status()
            return response.json()["choices"][0]["message"]["content"]
    except httpx.RequestError as e:
        print(f"Groq API request error: {e}")
        raise HTTPException(status_code=500, detail="Failed to connect to Groq API.")
    except (KeyError, IndexError) as e:
        print(f"Error parsing Groq API response: {e}")
        raise HTTPException(status_code=500, detail="Invalid response from Groq API.")

async def summarize_chat_for_bio(text: str, character_name: str) -> str:
    system_prompt = "You are a skilled biographer. Your task is to analyze the provided text, which contains messages from a person, and write a concise, descriptive, third-person biography about them. Focus on their personality, way of speaking, and any personal details mentioned."
    user_prompt = f"Based on the following messages from a person named '{character_name}', write their biography:\n\n---\n{text}\n---"
    return await call_groq_api(system_prompt, user_prompt)

async def extract_qna_from_chat(text: str) -> dict:
    system_prompt = "You are a data extraction specialist. Analyze the provided text and extract key personal details (like name, age, job, location, interests, etc.) into a valid JSON object. If a detail is not mentioned, do not include it. The keys should be in English. Respond ONLY with the raw JSON object and nothing else."
    user_prompt = f"Extract personal details from this text into a JSON object:\n\n---\n{text}\n---"
    json_string = await call_groq_api(system_prompt, user_prompt)
    try:
        clean_json_string = json_string.strip().replace("```json", "").replace("```", "")
        return json.loads(clean_json_string)
    except json.JSONDecodeError:
        print(f"Warning: Failed to decode JSON from Groq response: {json_string}")
        return {}

# --- Helper Functions ---
def convert_audio_to_text(audio_bytes: bytes) -> str:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio: # Common audio format
        temp_audio.write(audio_bytes)
        temp_audio.flush()
        audio_path = temp_audio.name
    try:
        result = whisper_model.transcribe(audio_path)
        transcribed_text = result['text']
    except Exception as e:
        print(f"Error during audio transcription: {e}")
        transcribed_text = "" # Return empty string or raise error
    finally:
        os.remove(audio_path)
    return transcribed_text
# ################################
from gtts import gTTS
import re
import tempfile

def detect_language(text: str) -> str:
    # البحث عن أي حرف عربي في النص
    if re.search(r'[\u0600-\u06FF]', text):
        return 'ar'  # عربي
    else:
        return 'en'  # إنجليزي

async def convert_text_to_audio_playai(text: str, language: str) -> str | None:
    """
    بتحول النص لصوت باستخدام نماذج PlayAI على منصة Groq.
    """
    global GROQ_API_KEY # للوصول للمفتاح المعرف في بداية الكود

    # 1. تحديد النموذج بناءً على اللغة
    # نستخدم النموذج المخصص للعربية إذا كانت اللغة 'ar'
    model_to_use = "playai-tts-arabic" if language == 'ar' else "playai-tts"

    print(f"Generating audio with PlayAI model: '{model_to_use}'...")

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_to_use,
        "input": text,
        "voice": "Nasser-PlayAI" # هذا الباراميتر قد يكون مطلوبًا لبعض النماذج، "aura-asteria-en" هو مثال شائع.
                                 # لنموذج العربية قد لا يكون ضروريًا، لكن من الجيد إبقاؤه.
    }

    try:
        # استخدام httpx لإرسال الطلب واستقبال الصوت كبيانات خام
        async with httpx.AsyncClient() as client:
            response = await client.post("https://api.groq.com/openai/v1/audio/speech", headers=headers, json=payload, timeout=60.0)

            # التأكد من نجاح الطلب، وإلا سيتم إطلاق خطأ
            response.raise_for_status()

            # 2. إنشاء ملف مؤقت لحفظ الصوت بصيغة mp3
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
                audio_path = f.name
                f.write(response.content) # كتابة محتوى الصوت الخام في الملف

            print(f"PlayAI audio file generated successfully at: {audio_path}")
            return audio_path

    except httpx.HTTPStatusError as e:
        # طباعة الخطأ الذي أرجعه السيرفر بوضوح
        print(f"❌ Error during PlayAI TTS API call: {e}")
        print(f"Response body: {e.response.text}")
        return None
    except Exception as e:
        print(f"❌ An unexpected error occurred during PlayAI conversion: {e}")
        return None

def convert_text_to_audio_gtts(text: str, language: str) -> str | None:
    """
    بتحول النص لصوت باستخدام مكتبة gTTS وبتحفظه في ملف MP3 مؤقت.
    """
    try:
        print(f"Generating audio with gTTS in language: '{language}'...")

        # 1. إنشاء كائن gTTS مع النص واللغة المحددة
        # slow=False عشان النطق يكون بالسرعة العادية
        tts = gTTS(text=text, lang=language, slow=False)

        # 2. إنشاء ملف مؤقت لحفظ الصوت بصيغة mp3
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
            audio_path = f.name
            tts.save(audio_path) # حفظ الصوت في الملف

        print(f"gTTS audio file generated successfully at: {audio_path}")
        return audio_path

    except Exception as e:
        print(f"❌ Error during gTTS conversion: {e}")
        return None


# تعديل دالة CoquiTTS
def convert_text_to_audio_xtts(text: str, language: str) -> str | None:
    try:
        print(f"Generating audio in language: {language} for text: '{text[:30]}...'")
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as f:
            audio_path = f.name

        # استدعاء الموديل بدون الصوت المرجعي
        # هو هيستخدم الصوت الافتراضي بتاعه للغة اللي هتحددها
        tts_model.tts_to_file(
            text=text,
            language=language,
            file_path=audio_path
        )
        print(f"Audio file generated successfully at: {audio_path}")
        return audio_path
    except Exception as e:
        print(f"Error during Coqui XTTS conversion: {e}")
        return None

@app.post("/preprocessing_whats_data")
async def preprocess_whatsapp_data(payload: WhatsAppPayload):
    try:
        user_number = payload.userId
        target_number = next((msg.from_user for msg in payload.messages if msg.from_user != user_number), None)
        if not target_number:
            raise HTTPException(status_code=400, detail="Could not identify the target user.")

        target_messages = [
            msg.body for msg in payload.messages if msg.from_user == target_number and msg.body and msg.body.strip()
        ]
        if not target_messages:
            raise HTTPException(status_code=400, detail="No messages found from the target user.")

        target_messages_text_full = "\n".join(target_messages)

        # 1. توليد السيرة الذاتية والأسئلة كالسابق
        bio_task = summarize_chat_for_bio(target_messages_text_full, payload.characterName)
        answers_task = extract_qna_from_chat(target_messages_text_full)
        generated_bio, generated_answers = await asyncio.gather(bio_task, answers_task)

        # 2. ***الإضافة الجديدة: اختيار عينة من الرسائل لتمثيل الأسلوب***
        # نختار الرسائل الأطول كمؤشر أولي على أنها تحتوي على معلومات وأسلوب
        target_messages.sort(key=len, reverse=True)
        sample_messages = target_messages[:25] # نأخذ أفضل 25 رسالة كعينة

        # 3. تجهيز البيانات للحفظ
        character_data_to_save = {
            "characterId": payload.characterId,
            "characterName": payload.characterName,
            "bio": generated_bio,
            "answers": generated_answers,
            "sample_messages": sample_messages  # <-- إضافة العينات
        }

        # 4. استدعاء الدالة المساعدة للحفظ
        # ملاحظة: سنحتاج لتعديل _save_character_data لتستقبل هذا الشكل الجديد
        # تم تحويل character_data_to_save إلى قاموس عادي بدلًا من Pydantic model مؤقتًا
        success = await _save_character_data(character_data_to_save)

        if not success:
            raise HTTPException(status_code=500, detail="AI processing successful, but saving data failed.")

        return {
            "status": "success",
            "message": f"Successfully preprocessed and stored data for {payload.characterName}.",
            "processedData": {
                "bio": generated_bio,
                "answers": generated_answers,
                "sample_messages": sample_messages # إرجاع العينات للتطبيق إذا لزم الأمر
            }
        }

    except HTTPException as e:
        raise e
    except Exception as e:
        print(f"An unexpected error in preprocess_whatsapp_data: {e}")
        raise HTTPException(status_code=500, detail=f"An internal error occurred: {str(e)}")

# --- Endpoints تسجيل الدخول لتيليجرام ---
@app.post("/telegram/send_code")
async def telegram_send_code(req: PhoneRequest):
    client = TelegramClient(StringSession(), API_ID, API_HASH)
    await client.connect()
    try:
        code_hash_data = await client.send_code_request(req.phone_number)
        pending_clients[req.phone_number] = (client, code_hash_data.phone_code_hash)
        return {"status": "code_sent"}
    except Exception as e:
        await client.disconnect()
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/telegram/login")
async def telegram_login(req: LoginRequest):
    if req.phone_number not in pending_clients:
        raise HTTPException(status_code=404, detail="Session not found. Please request a code first.")

    client, phone_code_hash = pending_clients[req.phone_number]
    try:
        await client.sign_in(req.phone_number, req.code, phone_code_hash=phone_code_hash)
    except SessionPasswordNeededError:
        if not req.password:
            return {"status": "password_required"}
        await client.sign_in(password=req.password)
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

    session_string = client.session.save()
    del pending_clients[req.phone_number]
    await client.disconnect()
    return {"status": "login_successful", "sessionString": session_string}

# --- Endpoints التعامل مع الشاتات ---
@app.get("/telegram/chats/{session_string}")
async def telegram_get_chats(session_string: str):
    async with TelegramClient(StringSession(session_string), API_ID, API_HASH) as client:
        chats = []
        dialogs = await client.get_dialogs(limit=70)
        for dialog in dialogs:
            if dialog.is_user and not dialog.entity.bot:
                name = dialog.name
                if hasattr(dialog.entity, 'first_name') and dialog.entity.first_name:
                    name = f"{dialog.entity.first_name} {dialog.entity.last_name or ''}".strip()

                # جلب الصورة كـ bytes وتحويلها إلى Base64 Data URI
                pic_base64 = None
                try:
                    buffer = io.BytesIO()
                    await client.download_profile_photo(dialog.entity, file=buffer)
                    if buffer.getbuffer().nbytes > 0:
                        encoded_pic = base64.b64encode(buffer.getvalue()).decode('utf-8')
                        pic_base64 = f"data:image/jpeg;base64,{encoded_pic}"
                except Exception:
                    pass # تجاهل إذا لم توجد صورة

                chats.append({"id": dialog.id, "name": name, "picUrl": pic_base64})
        return {"status": "success", "chats": chats}

@app.post("/telegram/process_chat/{session_string}")
async def telegram_process_chat(session_string: str, payload: ProcessChatRequest):
    async with TelegramClient(StringSession(session_string), API_ID, API_HASH) as client:
        try:
            # تدفئة ذاكرة الجلسة للتعرف على الكيانات
            print("Warming up session entity cache by fetching dialogs...")
            await client.get_dialogs()
            print("Entity cache is ready.")

            # 1. جلب الرسائل من المحادثة المحددة
            chat_id = payload.chatId
            messages = []
            user_entity = await client.get_entity(chat_id)
            me = await client.get_me()

            print(f"Fetching messages from chat with {user_entity.first_name}...")
            async for message in client.iter_messages(chat_id, limit=200): # يمكنك زيادة الحد إذا أردت
                if message.text:
                    # إضافة الرسالة مع تحديد المرسل والمستقبل
                    from_user = str(me.id) if message.out else str(user_entity.id)
                    messages.append({
                        "from_user": from_user,
                        "body": message.text
                        # يمكن إضافة باقي الحقول مثل timestamp إذا كانت مهمة للمعالجة
                    })

            # 2. استخلاص رسائل الشخصية المستهدفة فقط
            target_messages = [
                msg["body"] for msg in messages if msg["from_user"] != str(me.id) and msg["body"] and msg["body"].strip()
            ]
            if not target_messages:
                raise HTTPException(status_code=400, detail="No messages from the target user to process.")

            target_messages_text_full = "\n".join(target_messages)
            character_name = f"{user_entity.first_name} {user_entity.last_name or ''}".strip()

            # 3. استدعاء نماذج الذكاء الاصطناعي لتوليد البيانات
            print("Processing chat with AI to generate profile...")
            bio_task = summarize_chat_for_bio(target_messages_text_full, character_name)
            answers_task = extract_qna_from_chat(target_messages_text_full)
            generated_bio, generated_answers = await asyncio.gather(bio_task, answers_task)

            # 4. اختيار عينة من الرسائل لتمثيل الأسلوب
            target_messages.sort(key=len, reverse=True)
            sample_messages = target_messages[:25]

            # 5. تجهيز البيانات الكاملة للحفظ
            character_id_str = f"tg_{user_entity.id}" # معرف فريد لشخصيات تيليجرام
            character_data_to_save = {
                "characterId": character_id_str,
                "characterName": character_name,
                "bio": generated_bio,
                "answers": generated_answers,
                "sample_messages": sample_messages
            }

            # 6. ***استدعاء دالة الحفظ لإنشاء الذاكرة الدائمة (FAISS)***
            success = await _save_character_data(character_data_to_save)
            if not success:
                raise HTTPException(status_code=500, detail="AI processing was successful, but saving the data to server failed.")

            # 7. إرجاع بيانات الشخصية الكاملة للتطبيق للانتقال مباشرة إلى شاشة الدردشة
            print(f"✅ Successfully created and saved character '{character_name}' from Telegram.")
            return {
                "status": "success",
                "characterProfile": {
                    "id": character_id_str,
                    "name": character_name,
                    "bio": generated_bio,
                    "questionnaireAnswers": generated_answers, # اسم الحقل كما يتوقعه التطبيق
                    # قد تحتاج لإضافة رابط الصورة إذا كنت تريد عرضه في التطبيق بعد الإنشاء
                }
            }

        except Exception as e:
            import traceback
            print(f"An error occurred in telegram_process_chat: {e}")
            traceback.print_exc()
            raise HTTPException(status_code=500, detail=str(e))

async def _save_character_data(data: dict) -> bool: # نستقبل الآن قاموسًا مرنًا
    """
    دالة مساعدة تقوم بحفظ بيانات الشخصية وملفات FAISS بناءً على العينات الحقيقية.
    """
    try:
        char_id = data["characterId"]
        char_name = data["characterName"]
        bio = data["bio"]
        answers = data.get("answers", {}) # قد لا تكون موجودة في كل الحالات
        sample_messages = data.get("sample_messages", []) # العينات الفعلية

        char_path = BASE_CHARACTER_DATA_PATH / char_id
        char_path.mkdir(parents=True, exist_ok=True)

        # 1. حفظ profile.json (الآن يحتوي على العينات)
        profile_data = {
            "characterName": char_name,
            "bio": bio,
            "sample_messages": sample_messages # <-- حفظ العينات هنا
        }
        with open(char_path / "profile.json", "w", encoding="utf-8") as f:
            json.dump(profile_data, f, ensure_ascii=False, indent=4)

        # 2. حفظ questionnaire.json (إن وجد)
        with open(char_path / "questionnaire.json", "w", encoding="utf-8") as f:
            json.dump(answers, f, ensure_ascii=False, indent=4)

        # 3. ***تعديل جوهري: بناء الذاكرة من العينات الحقيقية والأسئلة***
        texts_to_index = []
        # إضافة العينات الفعلية كما هي للذاكرة
        if sample_messages:
            texts_to_index.extend(sample_messages)

        # إضافة الأسئلة والأجوبة كحقائق منفصلة
        for q_key, q_ans in answers.items():
            texts_to_index.append(f"When asked about '{q_key}', I answered: '{q_ans}'.")

        # 4. إنشاء/تحديث فهرس FAISS
        if texts_to_index:
            embeddings = embedding_model.encode(texts_to_index, show_progress_bar=True)
            if embeddings.ndim == 1:
                embeddings = np.asarray([embeddings])
            if embeddings.shape[0] > 0:
                char_faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
                char_faiss_index.add(embeddings)
                faiss.write_index(char_faiss_index, str(char_path / "index.faiss"))
                with open(char_path / "texts.json", "w", encoding="utf-8") as f:
                    json.dump(texts_to_index, f, ensure_ascii=False, indent=4)

        print(f"✅ Data for '{char_name}' (ID: {char_id}) saved successfully using real samples.")
        return True

    except Exception as e:
        import traceback
        print(f"❌ Error in _save_character_data for '{data.get('characterName', 'N/A')}': {e}")
        traceback.print_exc()
        return False

# Endpoint الأصلي يستدعي الدالة المساعدة
@app.post("/store_data")
async def store_data_endpoint(data: CharacterProfileData):
    # تحويل من Pydantic model إلى dict لتمريره للدالة المساعدة
    character_dict = data.dict()
    success = await _save_character_data(character_dict)
    if success:
        return {"status": "success", "message": f"Data for {data.characterName} stored/updated successfully!"}
    else:
        raise HTTPException(status_code=500, detail="Failed to store character data on the server.")


async def process_chat_or_call(
    characterId: str,
    user_message_text: str | None = None,
    user_audio_file: UploadFile | None = None,
    chat_history_str: str | None = None,
    is_call: bool = False
):
    char_path = BASE_CHARACTER_DATA_PATH / characterId
    if not char_path.exists() or not (char_path / "profile.json").exists():
        raise HTTPException(status_code=404, detail="Character not found or profile is missing.")

    # 1. تحميل كل بيانات الشخصية المتاحة
    try:
        with open(char_path / "profile.json", "r", encoding="utf-8") as f:
            profile = json.load(f)
        character_name = profile.get("characterName", "This Character")
        character_bio = profile.get("bio", "No bio available.")
        sample_messages = profile.get("sample_messages", [])

        # تحميل إجابات الاستبيان
        answers = {}
        if (char_path / "questionnaire.json").exists():
            with open(char_path / "questionnaire.json", "r", encoding="utf-8") as f:
                answers = json.load(f)

        # تحميل ذاكرة البحث RAG
        char_faiss_index = None
        char_stored_texts = []
        if (char_path / "index.faiss").exists():
            char_faiss_index = faiss.read_index(str(char_path / "index.faiss"))
            with open(char_path / "texts.json", "r", encoding="utf-8") as f:
                char_stored_texts = json.load(f)

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Could not load character data: {e}")

    # 2. تحديد رسالة المستخدم
    actual_user_message = ""
    if user_audio_file:
        audio_bytes = await user_audio_file.read()
        actual_user_message = convert_audio_to_text(audio_bytes)
    elif user_message_text:
        actual_user_message = user_message_text

    if not actual_user_message.strip():
        return {"reply": "...", "transcript": actual_user_message or None}

    # 3. بحث RAG (كما هو، لكنه الآن أكثر فعالية بفضل البيانات الأفضل)
    relevant_context = []
    if char_faiss_index and char_stored_texts:
        try:
            query_embedding = embedding_model.encode([actual_user_message])
            k_search = min(5, char_faiss_index.ntotal)
            if k_search > 0:
                _, indices = char_faiss_index.search(query_embedding, k=k_search)
                relevant_context = [char_stored_texts[i] for i in indices[0]]
        except Exception as e:
            print(f"RAG search failed for {characterId}: {e}")

    # 4. ***بناء الـ Prompt الاحترافي الجديد***

    # تحويل إجابات الاستبيان إلى نص واضح
    answers_text = "\n".join([f"- {key.replace('_', ' ').title()}: {value}" for key, value in answers.items()])

    # اختيار عينات عشوائية للأسلوب
    style_examples = "\n".join(np.random.choice(sample_messages, size=min(4, len(sample_messages)), replace=False)) if sample_messages else "No specific style examples available."

    # الـ System Prompt الجديد والمحسّن
    system_prompt_content = f"""You are an AI clone of '{character_name}'.

### المبادئ التوجيهية (Guiding Principles):
1.  **أجب على السؤال أولاً (Answer First):** هدفك الأساسي هو الإجابة على سؤال المستخدم بشكل مباشر ودقيق.
2.  **حافظ على الشخصية (Maintain Persona):** أثناء صياغة إجابتك، يجب أن تتقمص شخصية '{character_name}' بالكامل.
3.  **كن موجزًا (Be Concise):** حافظ على إجاباتك موجزة وذات صلة. تجنب المونولوجات الطويلة ما لم يطلب المستخدم ذلك.
4.  **استخدم ذاكرتك (Use Your Knowledge):** اعتمد على الحقائق الموجودة في "قاعدة المعرفة" و"السياق" للرد على الأسئلة الواقعية.

### تفاصيل الشخصية (Persona Details):
- **السيرة الذاتية (Bio):** {character_bio}
- **حقائق أساسية (Key Facts from Q&A):**
{answers_text if answers_text else "No specific facts available."}

### أمثلة على الأسلوب (Style Examples - Messages you have written):
{style_examples}
---
Now, begin the conversation. You are '{character_name}'.
"""

    # User Prompt (يبقى كما هو تقريبًا)
    chat_history_list = json.loads(chat_history_str) if chat_history_str else []
    formatted_chat_history = "\n".join([str(item) for item in chat_history_list])

    user_prompt_content = f"""Here is some relevant context from your memory that might help you answer:
<CONTEXT>
{chr(10).join(relevant_context) if relevant_context else "No specific context found."}
</CONTEXT>

Here is the recent conversation history:
<HISTORY>
{formatted_chat_history if formatted_chat_history else "This is the beginning of the conversation."}
</HISTORY>

The user you are talking to now asks:
"{actual_user_message}"

Based on your persona and all the information above, provide a direct and in-character response.
Your response (as {character_name}):"""

    # 5. استدعاء الموديل والعودة بالرد (باقي الكود لا يتغير)
    try:
        reply = await call_groq_api(system_prompt_content, user_prompt_content)
    except Exception as e:
        raise HTTPException(status_code=502, detail="Failed to get a response from the AI model.")

    if is_call:
        detected_language = detect_language(reply)
        audio_response_path = await convert_text_to_audio_playai(text=reply, language=detected_language)
        if audio_response_path:
            return FileResponse(audio_response_path, media_type="audio/mpeg", filename="response.mp3")
        else:
            raise HTTPException(status_code=500, detail="Failed to convert bot reply to audio with PlayAI.")
    else:
        return {"reply": reply, "transcript": actual_user_message or None}


@app.post("/chat")
async def chat_endpoint(
    characterId: str = Form(...),
    message: str = Form(None),
    file: UploadFile = File(None),
    chat_history_str: str = Form(None) # JSON string of chat history
):
    return await process_chat_or_call(
        characterId=characterId,
        user_message_text=message,
        user_audio_file=file,
        chat_history_str=chat_history_str,
        is_call=False
    )

@app.post("/call")
async def call_endpoint(
    characterId: str = Form(...),
    file: UploadFile = File(...) # Audio from user is mandatory for /call
):
    # For /call, chat_history_str is not explicitly part of the request in the suggestion.
    # If needed, it could be added similar to /chat.
    return await process_chat_or_call(
        characterId=characterId,
        user_audio_file=file,
        is_call=True
    )

# --- Server Startup (example) ---
async def main():
    config = uvicorn.Config(app="main:app", host="0.0.0.0", port=8000, log_level="info", reload=True)
    # Note: If you save this file as `main.py`, then `app="main:app"` is correct.
    # `reload=True` is good for development.
    server = uvicorn.Server(config)
    await server.serve()

if __name__ == "__main__":
    # This allows running with `python your_file_name.py`
    # The `nest_asyncio.apply()` at the top helps if uvicorn is run in an already running loop (e.g. Jupyter)
    # For typical CLI execution, uvicorn manages its own loop.
    # You can also run directly with: uvicorn main:app --host 0.0.0.0 --port 8000 --reload
    print("Starting server programmatically...")
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info") # Simpler config for direct run
    server = uvicorn.Server(config)
    # server.run() # This is synchronous for Server instance
    asyncio.run(server.serve()) # If you want to run the async serve()

ngrok public URL: https://1dfe-34-34-115-170.ngrok-free.app


100%|█████████████████████████████████████| 2.88G/2.88G [01:27<00:00, 35.1MiB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Starting server programmatically...


INFO:     Started server process [2749]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Data for 'M Ehab CS' (ID: 201093648472@c.us) saved successfully using real samples.
INFO:     196.137.163.45:0 - "POST /preprocessing_whats_data HTTP/1.1" 200 OK
INFO:     196.137.163.45:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     196.137.163.45:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     196.137.163.45:0 - "POST /chat HTTP/1.1" 200 OK
Generating audio with PlayAI model: 'playai-tts-arabic'...
PlayAI audio file generated successfully at: /tmp/tmp2javm433.mp3
INFO:     196.137.163.45:0 - "POST /call HTTP/1.1" 200 OK
Generating audio with PlayAI model: 'playai-tts-arabic'...
PlayAI audio file generated successfully at: /tmp/tmp3ug340re.mp3
INFO:     196.137.163.45:0 - "POST /call HTTP/1.1" 200 OK
